In [1]:
import pandas as pd
import pypsa
import glob
import os

In [2]:
iteration = 0

In [3]:
save_path = "resources/capacity_tables/0.h5"

In [4]:
climate_year = 1

network_list = glob.glob("resources/networks/0/cy{}_ty*.nc".format(climate_year))

In [5]:
networks = pd.Series(
    index = [i.split("ty")[1][:4] for i in network_list]
)

networks.index = networks.index.astype(int)

In [6]:
networks.index.name = "target_year"

In [7]:
for i, path in enumerate(network_list):
    networks.iloc[i] = pypsa.Network(path)

INFO:pypsa.network.io:New version 1.0.5 available! (Current: 1.0.3)
INFO:pypsa.network.io:Imported network 'Unnamed Network' has buses, generators, links, loads, storage_units
/tmp/ipykernel_2219074/1131893391.py:2: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'PyPSA Network 'Unnamed Network'' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

INFO:pypsa.network.io:New version 1.0.5 available! (Current: 1.0.3)
INFO:pypsa.network.io:Imported network 'Unnamed Network' has buses, generators, links, loads, storage_units
INFO:pypsa.network.io:New version 1.0.5 available! (Current: 1.0.3)
INFO:pypsa.network.io:Imported network 'Unnamed Network' has buses, generators, links, loads, storage_units
INFO:pypsa.network.io:New version 1.0.5 available! (Current: 1.0.3)
INFO:pypsa.network.io:Imported network 'Unnamed Network' has buses, generators, links, loads, storage_units


In [8]:
networks.sort_index(inplace=True)

In [14]:
generation_capacities = []
storage_capacities = []

In [15]:
for year in networks.index:
    generation_capacities.append(networks.loc[year].generators)
    storage_capacities.append(networks.loc[year].storage_units)

In [16]:
generation_capacities = pd.concat(generation_capacities, keys=networks.index)
storage_capacities = pd.concat(storage_capacities, keys=networks.index)

In [ ]:
generation_capacities = generation_capacities.reorder_levels(["name", "target_year"]).sort_index()
storage_capacities = storage_capacities.reorder_levels(["name", "target_year"]).sort_index()   

In [23]:
default_costs = pd.ExcelFile("data/Economic and technical investment parameters/Economic and technical investment parameters_Default.xlsx")

In [25]:
default_costs.sheet_names

['Read-me',
 'CAPEX',
 'VOM',
 'FOM',
 'Potential',
 'Economic Lifetime',
 'DSR Activation Price',
 'Activation Limit',
 'WACC',
 'Hurdle Premium',
 'De-Mothballing_Cost',
 'Mothballing_FOM',
 'LifeTime Extension',
 'LifeTime Extension_FOM']

In [34]:
default_capex = pd.read_excel(default_costs, "CAPEX", index_col=[0,1,2])

name         target_year
AT00 DSR 0   2028           DSR
             2030           DSR
             2033           DSR
             2035           DSR
BE00 DSR 1   2028           DSR
                           ... 
UKNI DSR 64  2030           DSR
             2033           DSR
             2035           DSR
UKNI DSR 65  2033           DSR
             2035           DSR
Name: carrier, Length: 262, dtype: object

In [40]:
default_capex = default_capex.drop("Retirement", level=2) # no entries

In [41]:
default_capex

2026   2028   2030   2035  \
Node    PEMMDB Technology Reference Technology                                 
Default Gas CCGT new      Expansion               889.0  889.0  889.0  889.0   
        H2 CCGT new       Expansion               977.9  977.9  977.9  977.9   
        Gas OCGT new      Expansion               522.0  522.0  522.0  522.0   
        H2 OCGT new       Expansion               600.3  600.3  600.3  600.3   
        Batteries         Battery Storage_2       926.2  926.2  926.2  866.7   
        DSR               Industry_DSR band 1      22.0   22.0   22.0   22.0   
                          Industry_DSR band 2      22.0   22.0   22.0   22.0   
                          Industry_DSR band 3      22.0   22.0   22.0   22.0   
                          Industry_DSR band 4      22.0   22.0   22.0   22.0   
                          Industry_DSR band 5      22.0   22.0   22.0   22.0   
                          Industry_DSR band 6      22.0   22.0   22.0   22.0   
                          Industry_DSR band 7      22.0   22.0   22.0   22.0   
                          Industry_DSR band 8      22.0   22.0   22.0   22.0   
                          Industry_DSR band 9      22.0   22.0   22.0   22.0   
                          Industry_DSR band 10     22.0   22.0   22.0   22.0   
                          Residential_DSR band 1  101.0  101.0  101.0  101.0   
                          Commercial_DSR band 1   240.0  240.0  240.0  240.0   

                                                        Additional Comments  
Node    PEMMDB Technology Reference Technology                               
Default Gas CCGT new      Expansion               Average from CONE Studies  
        H2 CCGT new       Expansion                        10% add on to NG  
        Gas OCGT new      Expansion               Average from CONE Studies  
        H2 OCGT new       Expansion                        15% add on to NG  
        Batteries         Battery Storage_2       Average from CONE Studies  
        DSR               Industry_DSR band 1     Average from CONE Studies  
                          Industry_DSR band 2     Average from CONE Studies  
                          Industry_DSR band 3     Average from CONE Studies  
                          Industry_DSR band 4     Average from CONE Studies  
                          Industry_DSR band 5     Average from CONE Studies  
                          Industry_DSR band 6     Average from CONE Studies  
                          Industry_DSR band 7     Average from CONE Studies  
                          Industry_DSR band 8     Average from CONE Studies  
                          Industry_DSR band 9     Average from CONE Studies  
                          Industry_DSR band 10    Average from CONE Studies  
                          Residential_DSR band 1  Average from CONE Studies  
                          Commercial_DSR band 1   Average from CONE Studies

In [ ]:
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [ ]:
generation_capacities.to_hdf(save_path, key="generators")
storage_capacities.to_hdf(save_path, key="storages")